<a href="https://colab.research.google.com/github/linhmai19/dsc-resilient-distributed-datasets-rdd-lab-online-ds-sp-000/blob/master/Resilient_Distributed_Datasets_(RDDs)_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

In [3]:
!pip install pyspark==2.4.0

In [4]:
# Create a Python list data of integers between 1 and 1000
data = list(range(1, 1001))
len(data)

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [5]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [6]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [7]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [9]:
# returns the total count of items in the RDD
rdd.count()

1000

In [10]:
# returns the first item in the RDD
rdd.first()

1

In [12]:
# returns the first n items in the RDD
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
# returns the top n items
rdd.top(5)

[1000, 999, 998, 997, 996]

In [ ]:
# returns everything from your RDD
rdd.collect()

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [17]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([1.48250253e-01, 1.81274180e+00, 3.65452573e-01, 2.46193307e+00,
       1.71815721e+00, 1.58353789e-01, 6.22781992e+00, 5.25387711e+00,
       1.09016542e+00, 2.21650097e+00, 4.16677686e+00, 1.07822107e+01,
       5.86465556e+00, 8.73343991e+00, 6.95142627e+00, 3.88702595e+00,
       1.24459125e+01, 5.08397397e+00, 5.96151257e+00, 1.00459154e+01,
       1.29560874e+01, 1.11597733e+00, 6.06627736e+00, 1.97104266e+01,
       2.19134986e+00, 2.45180875e+01, 1.70142650e+01, 1.32294071e+00,
       1.00310977e+01, 1.48874693e+01, 2.86278267e+01, 1.87278434e+01,
       2.56699186e+01, 1.21986146e+01, 3.20725477e+01, 2.01591999e+01,
       1.62395659e+01, 2.77165630e+01, 7.41274462e+00, 1.50851415e+01,
       3.40967579e+01, 1.28302594e+01, 1.85232328e+01, 4.11659760e+01,
       4.13459099e+01, 2.51336868e+01, 2.36010832e+01, 4.58513822e+01,
       1.25183739e+01, 2.05882638e+01, 1.98563938e+01, 2.09413545e+01,
       2.57446425e+01, 4.07536555e+01, 2.43079565e+01, 1.56323897e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [18]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)


[0.14825025266261527,
 1.8127417999822932,
 0.36545257287253574,
 2.4619330679004046]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [19]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [20]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.13639023244960605,
 1.6677224559837098,
 0.3362163670427329,
 2.2649784224683724,
 1.5807046363029114,
 0.14568548599175926,
 5.729594327192581,
 4.8335669433196475,
 1.0029521845559142,
 2.039180890498713]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [21]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [22]:
discounted.take(10)

[0.12275120920464544,
 1.500950210385339,
 0.3025947303384596,
 2.038480580221535,
 1.4226341726726202,
 0.13111693739258334,
 5.156634894473323,
 4.350210248987683,
 0.9026569661003228,
 1.8352628014488417]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [23]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[813.9535661951238,
 808.1500243870036,
 786.098037184167,
 752.9893453962015,
 744.963194106704,
 727.415906309577,
 715.1672086142426,
 714.2153955603035,
 709.762249416048,
 697.7832268180738,
 697.5037768320659,
 692.0140189685011,
 691.1335982151201,
 682.4848550764997,
 681.342754413888]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [24]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [25]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.14825025266261527, 0.12275120920464544), (1.8127417999822932, 1.500950210385339), (0.36545257287253574, 0.3025947303384596), (2.4619330679004046, 2.038480580221535), (1.7181572133727296, 1.4226341726726202), (0.15835378912147746, 0.13111693739258334), (6.227819920861501, 5.156634894473323), (5.2538771123039645, 4.350210248987683), (1.0901654179955589, 0.9026569661003228), (2.2165009679333836, 1.8352628014488417)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [26]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.14825025266261527, 0.12275120920464544, 1.8127417999822932, 1.500950210385339, 0.36545257287253574, 0.3025947303384596, 2.4619330679004046, 2.038480580221535, 1.7181572133727296, 1.4226341726726202]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [27]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)
# calculate total remaining in inventory 
selected_items.count()

264

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [28]:
selected_items.reduce(lambda x,y: x + y)

121489.54077650601

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [29]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(48, 304.695247544035),
 (48, 314.5387441032739),
 (17, 339.9048923173718),
 (48, 334.0972986168994),
 (42, 341.7431555637409),
 (7, 333.91146489303304),
 (21, 326.15055170962245)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [30]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(20, 4058.831986905205),
 (30, 957.7939902333285),
 (50, 2318.1081565656086),
 (40, 1072.4055697770577),
 (21, 3657.7008406794585),
 (11, 2517.3398143963036),
 (41, 4843.500435932669),
 (31, 1846.7830791242125),
 (1, 1206.894309423443),
 (42, 3829.8278370766093)]

In [31]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(33, 6839.2587861100255),
 (25, 6614.500211656902),
 (45, 5423.431017429708),
 (38, 5061.907415048679),
 (41, 4843.500435932669),
 (17, 4310.312340864783),
 (20, 4058.831986905205),
 (42, 3829.8278370766093),
 (21, 3657.7008406794585),
 (7, 3619.90732474349),
 (8, 3603.482395370413),
 (37, 3598.270158831065),
 (18, 3447.495044134939),
 (19, 3248.3100070096907),
 (48, 3218.358780578536),
 (24, 3211.387622803405),
 (32, 3154.817105517833),
 (44, 3084.888727323968),
 (35, 3082.8710908494395),
 (49, 2914.9513538879305),
 (3, 2859.6967373399266),
 (15, 2816.910409174924),
 (12, 2686.553108797154),
 (43, 2661.9389984447453),
 (39, 2563.7149977798144),
 (11, 2517.3398143963036),
 (28, 2487.7882992819814),
 (50, 2318.1081565656086),
 (6, 2125.9527133523807),
 (9, 1964.1061892419232),
 (31, 1846.7830791242125),
 (22, 1472.3587989340308),
 (46, 1450.5728802549374),
 (27, 1420.8007703663532),
 (13, 1352.3714379854275),
 (1, 1206.894309423443),
 (40, 1072.4055697770577),
 (30, 957.7939902333285),

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [32]:
# how many items were bought per user
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(19, 11),
 (49, 11),
 (36, 10),
 (8, 10),
 (50, 10),
 (2, 9),
 (17, 9),
 (11, 8),
 (25, 8),
 (22, 8),
 (33, 8),
 (21, 8),
 (5, 8),
 (9, 8),
 (35, 8),
 (6, 7),
 (24, 7),
 (13, 7),
 (42, 6),
 (7, 6),
 (26, 6),
 (47, 6),
 (14, 6),
 (30, 6),
 (46, 5),
 (34, 4),
 (18, 4),
 (38, 4),
 (40, 4),
 (41, 4),
 (37, 4),
 (4, 4),
 (16, 4),
 (27, 4),
 (39, 4),
 (23, 4),
 (48, 3),
 (32, 3),
 (1, 3),
 (20, 3),
 (12, 2),
 (28, 2),
 (15, 2),
 (45, 2),
 (44, 2),
 (43, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
